In [1]:
# Scientific imports

%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astroquery.mast import Observations
from astroquery.mast import Catalogs

# General imports

import csv, math, os, os.path, sys
import pandas as pd
import seaborn as sb
from sklearn import metrics
from IPython.display import display

sb.set()

Let's make a function such that I can check to see if the required headers to plot a LC are present in our dataset

In [9]:
def areAllHeadersPresent(heads):
    """
    A function to make sure that all headers required for plotting a Light Curve
    are present, and alert the user if any are missing (and which ones).
    
    Input parameter (heads) should be: heads=fits.getheader("file.fits")
    """
    
    # Generic Flag
    allgood = True
    
    # List of needed headers
    neededHeaders = ['TEFF', 'LOGG', 'TESSMAG',
                     'TPERIOD', 'TDUR', 'TEPOCH', 'TDEPTH',
                     'TIME', 'PHASE', 'LC_INIT', 'MODEL_INIT']
    
    # Loop through all headers and see if they are present using a Try/Except block
    for i in range (len(neededHeaders)):
        try:
            heads[neededHeaders[i]]
        except:
            #print("Header {} not present!".format(neededHeaders[i]))
            allgood = False
        #else:
        #    print("{}: {}".format(neededHeaders[i], heads[neededHeaders[i]]))
    return allgood

#areAllHeadersPresent(fitsHeaders)

In [10]:
# Reading which LC datafiles we have into a list

fitsList = []
fits_directories = [x[0] for x in os.walk('.')]

for fits_root, fits_dirs, fits_files in os.walk('.'):
    for fits_file in fits_files:
        fullpath = os.path.join(fits_root, fits_file)
        if (os.path.splitext(fullpath.lower())[1]).endswith('.fits'):
            fitsList.append(fullpath)
print("Number of FITS files: {}".format(len(fitsList)))
for num, i in enumerate(fitsList):
    print ("{}: {}".format(num, i))

Number of FITS files: 27
0: ./kplr010666592-2009131105131_llc.fits
1: ./mastDownload/IUE/lwp13058/lwp13058mxlo_vo.fits
2: ./mastDownload/TESS/tess2018206045859-s0001-0000000270501383-0120-s/tess2018206045859-s0001-0000000270501383-0120-s_lc.fits
3: ./mastDownload/TESS/tess2018206045859-s0001-0000000270501383-0120-s/tess2018206045859-s0001-0000000270501383-0120-s_tp.fits
4: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206045859-s0001-0000000097409519-0120-s_lc.fits
5: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206045859-s0001-0000000097409519-0120-s_tp.fits
6: ./mastDownload/TESS/tess2018206045859-s0001-0000000097409519-0120-s/tess2018206190142-s0001-s0001-0000000097409519-00106_dvt.fits
7: ./mastDownload/TESS/tess2018206045859-s0001-0000000099837626-0120-s/tess2018206045859-s0001-0000000099837626-0120-s_tp.fits
8: ./mastDownload/TESS/tess2018206045859-s0001-0000000099837626-0120-s/tess2018206045859-s0001-0000000099837626-0

In [12]:
#file = fitsList[0]
#check = fits.info(file)
#############################fitsHeaders = fits.getheader(file)
#fitsHeaders

Now that we have the list of FITS files, and our function to see if all required headers are present, let's loop through them all to see if *any* of them have **all** the required headers:

In [13]:
for i in range(len(fitsList)):
    tmp = fits.getheader(fitsList[i])
    print("{}:   {}".format(i, areAllHeadersPresent(tmp)))

0:   False
1:   False
2:   False
3:   False
4:   False
5:   False
6:   False
7:   False
8:   False
9:   False
10:   False
11:   False
12:   False
13:   False
14:   False
15:   False
16:   False
17:   False
18:   False
19:   False
20:   False
21:   False
22:   False
23:   False
24:   False
25:   False
26:   False


Oh... Huh. Joy of joys... I might need to re-assess what is a "required" header and which is not.

(One thing to note: Fields with a leading "T" are likely because we are looking for "TESS" based results; if we were using Kepler results, we'd probably wanna look for "KEPMAG", "KPERIOD", "KEPOCH" etc)

Next step - working out the _bare essential_ headers required to make a Light Curve!